## Импорт библиотек

In [1]:
# %pip install kornia

In [3]:
# %load_ext tensorboard
# %tensorboard --logdir runs

In [1]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.models.video import r3d_18
from torchvision.transforms import Compose, Resize, CenterCrop, Normalize, ToTensor
from sklearn.model_selection import train_test_split
from PIL import Image
import pandas as pd
from pathlib import Path
from torchvision.models.video import r3d_18, R3D_18_Weights
from tqdm import tqdm
import kornia as K
import kornia.augmentation as KAug
from torch.utils.tensorboard import SummaryWriter
import gc

2023-11-12 06:37:27.581858: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-12 06:37:27.625648: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-12 06:37:28.367225: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Создание классов из обучаемых данных

In [2]:
# Загрузка данных из CSV-файла
csv_data = pd.read_csv('classes.csv')

# Создание словаря для сопоставления названия класса и индекса класса
class_name_to_idx = pd.Series(csv_data.class_idx.values, index=csv_data.class_name).to_dict()

# Предположим, что ваши видеофайлы находятся в папке 'videos'
videos_path = Path('./dataset/videos')


In [3]:
# Генерируем список путей к файлам и соответствующих меток классов
file_paths = []
labels = []

for class_name, class_idx in class_name_to_idx.items():
    class_path = videos_path / class_name
    for video_file in class_path.iterdir():
        file_paths.append(str(video_file))
        labels.append(class_idx)

## Нахождение и настройка весов

In [4]:
#функция для извлечения кадров из видео
def extract_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()
    return frames

In [5]:
# Вычисление средних значений и стандартных отклонений
def compute_mean_std(videos_path):
    mean = np.zeros(3)
    std = np.zeros(3)
    n_frames = 0

    for video_path in tqdm(videos_path):
        frames = extract_frames(video_path)
        for frame in frames:
            mean += frame.mean(axis=(0, 1))
            std += frame.std(axis=(0, 1))
            n_frames += 1

    mean /= n_frames
    std /= n_frames
    return mean / 255, std / 255  # Нормализация значений

In [8]:
# mean, std = compute_mean_std(file_paths)
# print(f'Mean: {mean}, Std: {std}')

In [6]:
# Определите параметры
num_classes = 24  # Замените на количество классов в вашем датасете
num_epochs = 10
batch_size = 16
learning_rate = 1e-4
frame_count = 3  # Количество кадров в каждом видеосемпле

In [7]:
# Трансформации для предобработки видеофреймов
transform = Compose([
    Resize([128, 171], antialias=True),
    CenterCrop([112, 112]),
    ToTensor(),
    Normalize(mean=[0.35117388, 0.4031328,  0.41368235], std=[0.20939734, 0.21085491, 0.21630637])
])

## Обработка видеофайлов

### Класс создания датасета из видео

In [8]:
class VideoDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None, augmentation=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform
        self.augmentation = augmentation

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        frame_count = 3  # Задаем количество кадров
        video_path = self.file_paths[idx]
        cap = cv2.VideoCapture(video_path)
        frames = []
        try:
            while len(frames) < frame_count:
                ret, frame = cap.read()
                if not ret:
                    break
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = Image.fromarray(frame)
                
                if self.transform:
                    frame = self.transform(frame)
                if self.augmentation:
                    frame = self.augmentation(frame)
                frames.append(frame)
        finally:
            cap.release()

        # Обрезаем или дополняем список кадров до 8
        frames = frames[:frame_count] + [frames[-1]] * (frame_count - len(frames))

        frames = torch.stack(frames)
        label = torch.tensor(self.labels[idx])
        return frames, label
    

### Разделение на обучающий и валидационный датасеты

In [9]:
# Разделение на обучающий и валидационный датасеты
train_paths, valid_paths, train_labels, valid_labels = train_test_split(
    file_paths, labels, test_size=0.2, random_state=42, stratify=labels)

# Создание экземпляров датасета
train_dataset = VideoDataset(train_paths, train_labels, transform=transform)
valid_dataset = VideoDataset(valid_paths, valid_labels, transform=transform)

In [10]:
# Создание DataLoader'ов
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

## Создание модели и обучение

### Создание модели

In [11]:
# Создание модели
model = r3d_18(weights=R3D_18_Weights.KINETICS400_V1)
model.fc = nn.Linear(model.fc.in_features, num_classes)


# Перенесите модель на GPU, если доступно
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Функция потерь и оптимизатор
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


Downloading: "https://download.pytorch.org/models/r3d_18-b3b3357e.pth" to /tmp/xdg_cache/torch/hub/checkpoints/r3d_18-b3b3357e.pth
100%|██████████| 127M/127M [00:01<00:00, 67.2MB/s] 


### Обучение модели

In [15]:
# Обучение модели
writer = SummaryWriter('runs/experiment_name')

best_accuracy = 0.0  # Инициализация переменной для лучшей точности

for epoch in range(num_epochs):
    model.train()
    for i, (videos, labels) in enumerate(train_loader):
        videos = videos.to(device)
        labels = labels.to(device)

        # Обнуляем градиенты
        optimizer.zero_grad()
        
        # Прямой проход
        outputs = model(videos)
        
        # Расчет потерь
        loss = criterion(outputs, labels)
        
        # Обратное распространение
        loss.backward()
        
        # Оптимизация
        optimizer.step()
    
    # Валидация модели
    model.eval()
    correct = 0
    total = 0
    for videos, labels in valid_loader:
        videos = videos.to(device)
        labels = labels.to(device)
        outputs = model(videos)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        if (100 * correct / total) > best_accuracy:
            best_accuracy = 100 * correct / total
            torch.save(model.state_dict(), f'model_epoch_{epoch+1}.pth')
            print(f'Model saved at epoch {epoch+1}')
        
        # Для записи потерь обучения
    writer.add_scalar('Training Loss', loss.item(), epoch)

    # Для записи точности валидации
    writer.add_scalar('Validation Accuracy', 100 * correct / total, epoch)

    # Выводим статистику после каждой эпохи
    print(f'Epoch {epoch + 1}/{num_epochs}, Train Loss: {loss.item():.4f}, Validation Accuracy: {100 * correct / total:.2f}%')


writer.close()

Model saved at epoch 1
Model saved at epoch 1
Model saved at epoch 1
Model saved at epoch 1
Model saved at epoch 1
Model saved at epoch 1
Model saved at epoch 1
Model saved at epoch 1
Epoch 1/10, Train Loss: 1.3643, Validation Accuracy: 61.13%
Model saved at epoch 2
Epoch 2/10, Train Loss: 0.7674, Validation Accuracy: 68.22%
Model saved at epoch 3
Model saved at epoch 3
Epoch 3/10, Train Loss: 0.6865, Validation Accuracy: 72.87%
Model saved at epoch 4
Model saved at epoch 4
Model saved at epoch 4
Model saved at epoch 4
Epoch 4/10, Train Loss: 0.1500, Validation Accuracy: 73.08%
Model saved at epoch 5
Epoch 5/10, Train Loss: 0.2576, Validation Accuracy: 74.29%
Epoch 6/10, Train Loss: 0.1664, Validation Accuracy: 74.49%
Epoch 7/10, Train Loss: 0.1031, Validation Accuracy: 74.90%
Epoch 8/10, Train Loss: 0.0525, Validation Accuracy: 73.28%
Epoch 9/10, Train Loss: 0.0234, Validation Accuracy: 74.49%
Epoch 10/10, Train Loss: 0.0087, Validation Accuracy: 74.70%


In [23]:
# Сохранение модели
torch.save(model.state_dict(), 'action_recognition_model.pth')


## Проверка модели

In [12]:
model = r3d_18(weights=None)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load('action_recognition_model.pth'))
model.to(device)
model.eval()

VideoResNet(
  (stem): BasicStem(
    (0): Conv3d(3, 64, kernel_size=(3, 7, 7), stride=(1, 2, 2), padding=(1, 3, 3), bias=False)
    (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Sequential(
        (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (conv2): Sequential(
        (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (relu): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Sequential(
        (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1):

### Функции для распознавания действий в видео

In [23]:
import matplotlib.pyplot as plt
import cv2

def display_image(image):
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')  # Отключить оси координат для чистого отображения
    plt.show()

### один кадр в секунду

In [13]:
def process_frame(frame, model, transform, device, class_name_to_idx, frame_number, fps):
    # Обработка одного кадра в секунду
    if frame_number % int(fps) == 0:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = Image.fromarray(frame)
        frame = transform(frame).unsqueeze(0).to(device)

        with torch.no_grad():
            outputs = model(frame)
            _, predicted = torch.max(outputs, 1)
            class_name = [name for name, idx in class_name_to_idx.items() if idx == predicted.item()][0]
            timestamp = frame_number / fps
            
            return class_name, timestamp
    else:
        return None


### три кадра в секунду

In [14]:
import gc

def process_frame_group(frame_group, model, transform, device, class_name_to_idx, timestamp, fps):
    processed_frames = []
    for frame in frame_group:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = Image.fromarray(frame)
        frame = transform(frame)
        processed_frames.append(frame)

    # Стекирование кадров и добавление батч-размерности
    frames_tensor = torch.stack(processed_frames).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(frames_tensor)
        _, predicted = torch.max(outputs, 1)
        class_name = [name for name, idx in class_name_to_idx.items() if idx == predicted.item()][0]
    
    del frames_tensor
    gc.collect()
    
    return class_name, timestamp


### Многопоточная обработка видео

In [19]:
import concurrent.futures

def process_video_parallel(video_path, model, transform, device, class_name_to_idx):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    action_timestamps = {class_name: [] for class_name in class_name_to_idx.keys()}
    
    max_workers = 4  # Ограничение количества одновременных задач
    
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()

    # Выборка трех кадров каждую секунду
    frames_to_process = [frames[i:i+3] for i in range(0, len(frames), int(fps)) if i+3 <= len(frames)]
    del frames
    gc.collect()
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_frame = {executor.submit(process_frame_group, frame_group, model, transform, device, class_name_to_idx, idx / fps, fps): idx for idx, frame_group in enumerate(frames_to_process)}
        for future in concurrent.futures.as_completed(future_to_frame):
            result = future.result()
            if result is not None:
                class_name, timestamp = result
                print(result)
                display_image(frames_to_process[future_to_frame[future]][0])
                action_timestamps[class_name].append(timestamp)
            del result
    del frames_to_process
    gc.collect()  # Явный вызов сборщика мусора
    return action_timestamps

### Однопоточная обработка видео

In [20]:
def process_video(video_path, model, transform, device, class_name_to_idx):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    action_timestamps = {class_name: [] for class_name in class_name_to_idx.keys()}

    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()

    # Выборка трех кадров каждую секунду
    frames_to_process = [frames[i:i+3] for i in range(0, len(frames), int(fps)) if i+3 <= len(frames)]
    del frames
    gc.collect()
    print('Обработка видео...')

    for idx, frame_group in enumerate(frames_to_process):
        result = process_frame_group(frame_group, model, transform, device, class_name_to_idx, idx / fps, fps)
        if result is not None:
            class_name, timestamp = result
            print(result)
            display_image(frame_group[0])  # Отображение первого кадра в группе
            action_timestamps[class_name].append(timestamp)

    return action_timestamps


### Обработка папки с видеоданными

In [21]:
videos_path = Path('./test_data/test_dataset/')
video_files = [file for file in videos_path.iterdir() if file.suffix in ['.mp4', '.avi']]
for video_path in tqdm(video_files):
    try:
        video_path_str = str(video_path)
        timestamps = process_video(video_path_str, model, transform, device, class_name_to_idx)
        for action, times in timestamps.items():
            if times is not None:
                print(f"Действие '{action}' обнаружено в следующих временных метках: {times} в видео {video_path}")
    except Exception as e:
        print(f"Ошибка при обработке видео {video_path}: {e}")

  0%|          | 0/1397 [00:00<?, ?it/s]

Обработка видео...


  0%|          | 1/1397 [00:00<11:46,  1.98it/s]

('shoot_ball', 0.0)
Ошибка при обработке видео test_data/test_dataset/001afca4-f375-43eb-93be-67bff5046313.avi: name 'display_image' is not defined
Обработка видео...


  0%|          | 2/1397 [00:01<11:44,  1.98it/s]

...

('shoot_ball', 0.0)
Ошибка при обработке видео test_data/test_dataset/ff7e4e3f-9598-4126-b28a-4a6f625bafbf.avi: name 'display_image' is not defined
Обработка видео...


100%|█████████▉| 1396/1397 [11:33<00:00,  1.97it/s]

('golf', 0.0)
Ошибка при обработке видео test_data/test_dataset/ff89118e-6c35-461a-82df-8bd5937a3413.avi: name 'display_image' is not defined
Обработка видео...


100%|██████████| 1397/1397 [11:34<00:00,  2.01it/s]

('pullup', 0.0)
Ошибка при обработке видео test_data/test_dataset/fff5714b-5592-4851-8e16-cfb7b8a653e1.avi: name 'display_image' is not defined


### Обработка одного видео

In [ ]:
video_path = 'zxc.mp4'
Убедитесь, что модель находится в режиме .eval() и загружена на устройство перед вызовом этой функции
timestamps = process_video_parallel(video_path, model, transform, device, class_name_to_idx)